## Module 1: Intro

In [9]:
import minsearch
import json

from openai import OpenAI

In [10]:
with open("documents.json") as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [11]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}

    result = index.search(
        query=query,
        filter_dict={"course": "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5
    )

    return result

In [18]:
def build_prompt(query, search_result):
    prompt_template = """
        You're a course teaching assistant. Answer the question based on the context from the FAQ database.
        Use only the facts from the context when answering the questions.
        QUESTION: {question}
        CONTEXT:
            {context}
    """.strip()

    context = ""
    for doc in search_result:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [22]:
def llm(prompt):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [20]:
def rag(query):
    search_result = search(query)
    prompt = build_prompt(query, search_result)
    return llm(prompt)

In [21]:
rag("how to run kafka?")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}